In [0]:
#Installing the Shaply library 
pip install Shapely

Python interpreter will be restarted.
Collecting Shapely
 Using cached Shapely-1.7.1-cp37-cp37m-manylinux1_x86_64.whl (1.0 MB)
Installing collected packages: Shapely
Successfully installed Shapely-1.7.1
Python interpreter will be restarted.

In [0]:
#install Dask library 
pip install dask

Python interpreter will be restarted.
Collecting dask
 Using cached dask-2.30.0-py3-none-any.whl (848 kB)
Processing /root/.cache/pip/wheels/5e/03/1e/e1e954795d6f35dfc7b637fe2277bff021303bd9570ecea653/PyYAML-5.3.1-cp37-cp37m-linux_x86_64.whl
Installing collected packages: pyyaml, dask
Successfully installed dask-2.30.0 pyyaml-5.3.1
Python interpreter will be restarted.

In [0]:
#dask dataframe pachage
ip install "dask[dataframe]"

Python interpreter will be restarted.
Requirement already satisfied: dask[dataframe] in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9deccaf1-8cfc-45bd-a180-87ff2ef77ba5/lib/python3.7/site-packages (2.30.0)
Requirement already satisfied: pyyaml in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9deccaf1-8cfc-45bd-a180-87ff2ef77ba5/lib/python3.7/site-packages (from dask[dataframe]) (5.3.1)
Requirement already satisfied: pandas>=0.23.0; extra == "dataframe" in /databricks/python3/lib/python3.7/site-packages (from dask[dataframe]) (1.0.1)
Collecting fsspec>=0.6.0; extra == "dataframe"
 Using cached fsspec-0.8.4-py3-none-any.whl (91 kB)
Collecting partd>=0.3.10; extra == "dataframe"
 Using cached partd-1.1.0-py3-none-any.whl (19 kB)
Requirement already satisfied: numpy>=1.13.0; extra == "dataframe" in /databricks/python3/lib/python3.7/site-packages (from dask[dataframe]) (1.18.1)
Collecting toolz>=0.8.2; extra == "dataframe"
 Using cached toolz-0.11.1-py3-none-any.whl (55 kB)
Requirement already satisfied: python-dateutil>=2.6.1 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.23.0; extra == "dataframe"->dask[dataframe]) (2.8.1)
Requirement already satisfied: pytz>=2017.2 in /databricks/python3/lib/python3.7/site-packages (from pandas>=0.23.0; extra == "dataframe"->dask[dataframe]) (2019.3)
Processing /root/.cache/pip/wheels/7d/21/ce/56f01c644a11bde5d09ecae16d9b5e9d7e988187624fd28fec/locket-0.2.0-py3-none-any.whl
Requirement already satisfied: six>=1.5 in /databricks/python3/lib/python3.7/site-packages (from python-dateutil>=2.6.1->pandas>=0.23.0; extra == "dataframe"->dask[dataframe]) (1.14.0)
Installing collected packages: fsspec, locket, toolz, partd
Successfully installed fsspec-0.8.4 locket-0.2.0 partd-1.1.0 toolz-0.11.1
Python interpreter will be restarted.

In [0]:
#Take copy of the Dataset to local desk of data bricks to be accessed by python  
%fs cp dbfs:/FileStore/tables/sample.csv file:/databricks/driver/data.csv

res0: Boolean = true

In [0]:
#  tacke copy of Geojson file to local desk of data bricks to be accessed by python
%fs cp dbfs:/FileStore/tables/nyc_boroughs.geojson file:/databricks/driver/NY_neighborhoods.geojson

res1: Boolean = true

In [0]:
#Import all libraries that is going to be used 

import json
import time
import datetime
import numpy as np
import dask.dataframe as dd
from shapely.geometry import Point, Polygon
import pandas as pd
import dask.dataframe as dd
import dask.multiprocessing
import dask.threaded
def neighborhood_name_adder(data, geo_data, column_longitude, column_latitude, new_column_name):
    """
    data : Dask data frame of taxi trips
    geo_data: GeoJson containing the shape of each neighborhood
    column_longitude: Name of the longitude column in the dataset
    column_latitude: Name of the latitude column in the dataset
    new_column_name: Name of the new column (i.e. `pickup_neighborhood`)
    """
    from matplotlib import path
    import numpy as np
       
    
    lat = data[column_latitude]      # selecting longitude from dask dataframe
    lon = data[column_longitude]     # selecting latitude from dask dataframe
    
    data[new_column_name] = np.zeros(len(data)) # iniitalizing new column
    
    # The loop below goes through every neighborhood in the GeoJson file and maps the neighborhood name to the new column if a trip is within that neighborhood

    
    for feature in geo_data['features']:     # looping through every neighborhood in GeoJson file, and mapping all trips
        coords = feature['geometry']['coordinates'][0]
        p = path.Path(coords)
        index = p.contains_points(list(zip(lon, lat)))
        data[new_column_name].loc[index] = [str(feature['properties']['borough'])]*np.sum(index)

    
    return(data)
  
  

taxi_trips = pd.read_csv('/databricks/driver/data.csv')  # loading in data using dask

print ("old data set columns are: ", taxi_trips.columns)

geo_data =  json.load(open('/databricks/driver/NY_neighborhoods.geojson'))   # GeoJson file

taxi_trips_updated = neighborhood_name_adder(taxi_trips, geo_data, 'pickup_longitude', 'pickup_latitude', 'Pickup_borhood')
taxi_trips_updated1 = neighborhood_name_adder(taxi_trips_updated, geo_data, 'dropoff_longitude', 'dropoff_latitude', 'dropoff_borhood')


print ("updated data set columns are: ", taxi_trips_updated1.columns)

old data set columns are: Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
 'passenger_count', 'trip_time_in_secs', 'trip_distance',
 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
 'dropoff_latitude'],
 dtype='object')
/databricks/python/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
 self._setitem_with_indexer(indexer, value)
updated data set columns are: Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
 'passenger_count', 'trip_time_in_secs', 'trip_distance',
 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
 'dropoff_latitude', 'Pickup_borhood', 'dropoff_borhood'],
 dtype='object')

In [0]:
#writing the results of the above function to csv file on the local desk 
taxi_trips_updated1.to_csv (r'/databricks/driver/export_dataframe.csv', index = False, header=True)

In [0]:
#reading the above csv file 
taxi_trips = dd.read_csv('/databricks/driver/export_dataframe.csv')

In [0]:
#print the columns to check that the file is working properly 
print ("old data set columns are: ", taxi_trips.columns)

old data set columns are: Index(['medallion', 'hack_license', 'vendor_id', 'rate_code',
 'store_and_fwd_flag', 'pickup_datetime', 'dropoff_datetime',
 'passenger_count', 'trip_time_in_secs', 'trip_distance',
 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude',
 'dropoff_latitude', 'Pickup_borhood', 'dropoff_borhood'],
 dtype='object')

In [0]:
#copy the above created file to  data bricks file system  
%fs cp file:/databricks/driver/export_dataframe.csv dbfs:/FileStore/tables/export_dataframe_py_to_sp.csv

res2: Boolean = true

In [0]:
#Import the required libraries 

import sys
from pyspark.sql.window import Window
import pyspark.sql.functions as func
from pyspark.sql.functions import unix_timestamp
from pyspark.sql.functions import col, to_date
from pyspark.sql.window import Window
from pyspark.sql.functions import desc,max,col,sum,rank,asc,collect_list
from pyspark.sql.functions import lag
from pyspark.sql import SparkSession


In [0]:
# start spark 
spark = SparkSession.builder.getOrCreate()
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
# Register the csv file 
NYTaxi = spark.read.format("csv").option("header", "true").load("/FileStore/tables/export_dataframe_py_to_sp.csv")
NYTaxi.createOrReplaceTempView("dfTable")

In [0]:
#Check the DataFarme schema 
NYTaxi.printSchema()

root
-- medallion: string (nullable = true)
-- hack_license: string (nullable = true)
-- vendor_id: string (nullable = true)
-- rate_code: string (nullable = true)
-- store_and_fwd_flag: string (nullable = true)
-- pickup_datetime: string (nullable = true)
-- dropoff_datetime: string (nullable = true)
-- passenger_count: string (nullable = true)
-- trip_time_in_secs: string (nullable = true)
-- trip_distance: string (nullable = true)
-- pickup_longitude: string (nullable = true)
-- pickup_latitude: string (nullable = true)
-- dropoff_longitude: string (nullable = true)
-- dropoff_latitude: string (nullable = true)
-- Pickup_borhood: string (nullable = true)
-- dropoff_borhood: string (nullable = true)

In [0]:
#show 8 records of the dataset 
NYTaxi.limit(8).toPandas()

,medallion,hack_license,vendor_id,rate_code,store_and_fwd_flag,pickup_datetime,dropoff_datetime,passenger_count,trip_time_in_secs,trip_distance,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,Pickup_borhood,dropoff_borhood
0,89D227B655E5C82AECF13C3F540D4CF4,BA96DE419E711691B9445D6A6307C170,CMT,1,N,2013-01-01 15:11:48,2013-01-01 15:18:10,4,382,1.0,-73.978165,40.757977000000004,-73.989838,40.751171,Manhattan,Manhattan
1,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-06 00:18:35,2013-01-06 00:22:54,1,259,1.5,-74.006683,40.731781,-73.994499,40.750659999999996,Manhattan,Manhattan
2,0BD7C8F5BA12B88E0B67BED28BEA73D8,9FD8F69F0804BDB5549F40E9DA1BE472,CMT,1,N,2013-01-05 18:49:41,2013-01-05 18:54:23,1,282,1.1,-74.004707,40.737770000000005,-74.009834,40.726002,Manhattan,Manhattan
3,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:54:15,2013-01-07 23:58:20,2,244,0.7,-73.974602,40.759945,-73.984734,40.759388,Manhattan,Manhattan
4,DFD2202EE08F7A8DC9A57B02ACB81FE2,51EE87E3205C985EF8431D850C786310,CMT,1,N,2013-01-07 23:25:03,2013-01-07 23:34:24,1,560,2.1,-73.97625,40.748528,-74.002586,40.747868,Manhattan,Manhattan
5,20D9ECB2CA0767CF7A01564DF2844A3E,598CCE5B9C1918568DEE71F43CF26CD2,CMT,1,N,2013-01-07 15:27:48,2013-01-07 15:38:37,1,648,1.7,-73.96674300000001,40.764252,-73.983322,40.743763,Manhattan,Manhattan
6,496644932DF3932605C22C7926FF0FE0,513189AD756FF14FE670D10B92FAF04C,CMT,1,N,2013-01-08 11:01:15,2013-01-08 11:08:14,1,418,0.8,-73.995804,40.743977,-74.007416,40.744343,Manhattan,Manhattan
7,0B57B9633A2FECD3D3B1944AFC7471CF,CCD4367B417ED6634D986F573A552A62,CMT,1,N,2013-01-07 12:39:18,2013-01-07 13:10:56,3,1898,10.7,-73.989937,40.756775,-73.86525,40.77063,Manhattan,Queens


In [0]:
#check the number of records in the dataset  
NYTaxi.selectExpr("*").count()

Out[46]: 99999

In [0]:
#Cleaning the Dataset 
NYTaxi = NYTaxi.where("dropoff_borhood != '0.0'").select('*')

In [0]:
#select the dataset with borhood 
NYTaxi = NYTaxi.where("Pickup_borhood != '0.0'").select(NYTaxi['hack_license'],NYTaxi['pickup_datetime'],NYTaxi['dropoff_datetime'],NYTaxi['Pickup_borhood'],NYTaxi['dropoff_borhood'])

In [0]:
#check the Data set number records after cleaning 
NYTaxi.selectExpr("*").count()

Out[49]: 97507

In [0]:
NYTaxi.where("Pickup_borhood != dropoff_borhood").selectExpr("*").count()

Out[50]: 11433

In [0]:
NYTaxi.where("Pickup_borhood == dropoff_borhood").selectExpr("*").count()

Out[51]: 86074

In [0]:
#convert both pickup and dropoff time to Unix TimeStamp 
NYTaxi = NYTaxi.withColumn('pickup_unix', unix_timestamp('pickup_datetime'))
NYTaxi = NYTaxi.withColumn('dropoff_unix', unix_timestamp('dropoff_datetime'))


In [0]:
#check the created cloumns 
NYTaxi.limit(10).toPandas()

,hack_license,pickup_datetime,dropoff_datetime,Pickup_borhood,dropoff_borhood,pickup_unix,dropoff_unix
0,BA96DE419E711691B9445D6A6307C170,2013-01-01 15:11:48,2013-01-01 15:18:10,Manhattan,Manhattan,1357053108,1357053490
1,9FD8F69F0804BDB5549F40E9DA1BE472,2013-01-06 00:18:35,2013-01-06 00:22:54,Manhattan,Manhattan,1357431515,1357431774
2,9FD8F69F0804BDB5549F40E9DA1BE472,2013-01-05 18:49:41,2013-01-05 18:54:23,Manhattan,Manhattan,1357411781,1357412063
3,51EE87E3205C985EF8431D850C786310,2013-01-07 23:54:15,2013-01-07 23:58:20,Manhattan,Manhattan,1357602855,1357603100
4,51EE87E3205C985EF8431D850C786310,2013-01-07 23:25:03,2013-01-07 23:34:24,Manhattan,Manhattan,1357601103,1357601664
5,598CCE5B9C1918568DEE71F43CF26CD2,2013-01-07 15:27:48,2013-01-07 15:38:37,Manhattan,Manhattan,1357572468,1357573117
6,513189AD756FF14FE670D10B92FAF04C,2013-01-08 11:01:15,2013-01-08 11:08:14,Manhattan,Manhattan,1357642875,1357643294
7,CCD4367B417ED6634D986F573A552A62,2013-01-07 12:39:18,2013-01-07 13:10:56,Manhattan,Queens,1357562358,1357564256
8,1DA2F6543A62B8ED934771661A9D2FA0,2013-01-07 18:15:47,2013-01-07 18:20:47,Manhattan,Manhattan,1357582547,1357582847
9,CD2F522EEE1FF5F5A8D8B679E23576B3,2013-01-07 15:33:28,2013-01-07 15:49:26,Manhattan,Manhattan,1357572808,1357573766


In [0]:
#create window for each hack_license and order it by pickup timestamp 
windowSpec = Window.partitionBy(NYTaxi['hack_license']).orderBy(NYTaxi['pickup_unix'].asc())

In [0]:
#create column for the previouse dropoff time stamp 
NYTaxi = NYTaxi.withColumn("lag_dropoff_unix",lag("dropoff_unix",1).over(windowSpec))


In [0]:
NYTaxi.show()

+--------------------+-------------------+-------------------+--------------+---------------+-----------+------------+----------------+
 hack_license| pickup_datetime| dropoff_datetime|Pickup_borhood|dropoff_borhood|pickup_unix|dropoff_unix|lag_dropoff_unix|
+--------------------+-------------------+-------------------+--------------+---------------+-----------+------------+----------------+
006313464EC98A24B...|2013-01-13 07:36:00|2013-01-13 07:48:00| Manhattan| Manhattan| 1358062560| 1358063280| null|
006313464EC98A24B...|2013-01-13 08:16:00|2013-01-13 08:20:00| Manhattan| Manhattan| 1358064960| 1358065200| 1358063280|
006313464EC98A24B...|2013-01-13 08:51:00|2013-01-13 08:53:00| Manhattan| Manhattan| 1358067060| 1358067180| 1358065200|
006313464EC98A24B...|2013-01-13 10:03:00|2013-01-13 10:07:00| Manhattan| Manhattan| 1358071380| 1358071620| 1358067180|
006313464EC98A24B...|2013-01-13 11:31:00|2013-01-13 11:35:00| Manhattan| Manhattan| 1358076660| 1358076900| 1358071620|
006313464EC98A24B...|2013-01-13 13:04:00|2013-01-13 13:31:00| Manhattan| Manhattan| 1358082240| 1358083860| 1358076900|
006313464EC98A24B...|2013-01-13 13:38:00|2013-01-13 13:40:00| Manhattan| Manhattan| 1358084280| 1358084400| 1358083860|
006313464EC98A24B...|2013-01-13 13:42:00|2013-01-13 13:45:00| Manhattan| Manhattan| 1358084520| 1358084700| 1358084400|
006313464EC98A24B...|2013-01-13 13:52:00|2013-01-13 13:59:00| Manhattan| Manhattan| 1358085120| 1358085540| 1358084700|
006313464EC98A24B...|2013-01-13 14:08:00|2013-01-13 14:17:00| Manhattan| Manhattan| 1358086080| 1358086620| 1358085540|
006313464EC98A24B...|2013-01-13 14:17:00|2013-01-13 14:29:00| Manhattan| Manhattan| 1358086620| 1358087340| 1358086620|
006313464EC98A24B...|2013-01-13 14:49:00|2013-01-13 14:59:00| Manhattan| Manhattan| 1358088540| 1358089140| 1358087340|
006313464EC98A24B...|2013-01-13 15:04:00|2013-01-13 15:11:00| Manhattan| Manhattan| 1358089440| 1358089860| 1358089140|
006313464EC98A24B...|2013-01-13 15:21:00|2013-01-13 15:29:00| Manhattan| Manhattan| 1358090460| 1358090940| 1358089860|
006313464EC98A24B...|2013-01-13 15:39:00|2013-01-13 15:59:00| Manhattan| Manhattan| 1358091540| 1358092740| 1358090940|
006313464EC98A24B...|2013-01-13 16:02:00|2013-01-13 16:15:00| Manhattan| Manhattan| 1358092920| 1358093700| 1358092740|
006313464EC98A24B...|2013-01-13 16:16:00|2013-01-13 16:24:00| Manhattan| Manhattan| 1358093760| 1358094240| 1358093700|
006313464EC98A24B...|2013-01-13 16:24:00|2013-01-13 16:35:00| Manhattan| Manhattan| 1358094240| 1358094900| 1358094240|
006313464EC98A24B...|2013-01-13 17:27:00|2013-01-13 17:37:00| Manhattan| Manhattan| 1358098020| 1358098620| 1358094900|
006313464EC98A24B...|2013-01-13 17:59:00|2013-01-13 18:06:00| Manhattan| Manhattan| 1358099940| 1358100360| 1358098620|
+--------------------+-------------------+-------------------+--------------+---------------+-----------+------------+----------------+
only showing top 20 rows

In [0]:
# filter the trips to exclude the first trip  
NYTaxi = NYTaxi.where("lag_dropoff_unix IS NOT NULL").select('*')

In [0]:
NYTaxi.show()

+--------------------+-------------------+-------------------+--------------+---------------+-----------+------------+----------------+
 hack_license| pickup_datetime| dropoff_datetime|Pickup_borhood|dropoff_borhood|pickup_unix|dropoff_unix|lag_dropoff_unix|
+--------------------+-------------------+-------------------+--------------+---------------+-----------+------------+----------------+
006313464EC98A24B...|2013-01-13 08:16:00|2013-01-13 08:20:00| Manhattan| Manhattan| 1358064960| 1358065200| 1358063280|
006313464EC98A24B...|2013-01-13 08:51:00|2013-01-13 08:53:00| Manhattan| Manhattan| 1358067060| 1358067180| 1358065200|
006313464EC98A24B...|2013-01-13 10:03:00|2013-01-13 10:07:00| Manhattan| Manhattan| 1358071380| 1358071620| 1358067180|
006313464EC98A24B...|2013-01-13 11:31:00|2013-01-13 11:35:00| Manhattan| Manhattan| 1358076660| 1358076900| 1358071620|
006313464EC98A24B...|2013-01-13 13:04:00|2013-01-13 13:31:00| Manhattan| Manhattan| 1358082240| 1358083860| 1358076900|
006313464EC98A24B...|2013-01-13 13:38:00|2013-01-13 13:40:00| Manhattan| Manhattan| 1358084280| 1358084400| 1358083860|
006313464EC98A24B...|2013-01-13 13:42:00|2013-01-13 13:45:00| Manhattan| Manhattan| 1358084520| 1358084700| 1358084400|
006313464EC98A24B...|2013-01-13 13:52:00|2013-01-13 13:59:00| Manhattan| Manhattan| 1358085120| 1358085540| 1358084700|
006313464EC98A24B...|2013-01-13 14:08:00|2013-01-13 14:17:00| Manhattan| Manhattan| 1358086080| 1358086620| 1358085540|
006313464EC98A24B...|2013-01-13 14:17:00|2013-01-13 14:29:00| Manhattan| Manhattan| 1358086620| 1358087340| 1358086620|
006313464EC98A24B...|2013-01-13 14:49:00|2013-01-13 14:59:00| Manhattan| Manhattan| 1358088540| 1358089140| 1358087340|
006313464EC98A24B...|2013-01-13 15:04:00|2013-01-13 15:11:00| Manhattan| Manhattan| 1358089440| 1358089860| 1358089140|
006313464EC98A24B...|2013-01-13 15:21:00|2013-01-13 15:29:00| Manhattan| Manhattan| 1358090460| 1358090940| 1358089860|
006313464EC98A24B...|2013-01-13 15:39:00|2013-01-13 15:59:00| Manhattan| Manhattan| 1358091540| 1358092740| 1358090940|
006313464EC98A24B...|2013-01-13 16:02:00|2013-01-13 16:15:00| Manhattan| Manhattan| 1358092920| 1358093700| 1358092740|
006313464EC98A24B...|2013-01-13 16:16:00|2013-01-13 16:24:00| Manhattan| Manhattan| 1358093760| 1358094240| 1358093700|
006313464EC98A24B...|2013-01-13 16:24:00|2013-01-13 16:35:00| Manhattan| Manhattan| 1358094240| 1358094900| 1358094240|
006313464EC98A24B...|2013-01-13 17:27:00|2013-01-13 17:37:00| Manhattan| Manhattan| 1358098020| 1358098620| 1358094900|
006313464EC98A24B...|2013-01-13 17:59:00|2013-01-13 18:06:00| Manhattan| Manhattan| 1358099940| 1358100360| 1358098620|
006313464EC98A24B...|2013-01-13 19:21:00|2013-01-13 19:29:00| Manhattan| Manhattan| 1358104860| 1358105340| 1358100360|
+--------------------+-------------------+-------------------+--------------+---------------+-----------+------------+----------------+
only showing top 20 rows

In [0]:
# add new columns for the time between trips 
NYTaxi = NYTaxi.withColumn("time_Between_trips",( NYTaxi["pickup_unix"] - NYTaxi["lag_dropoff_unix"] ) )
NYTaxi.limit(5).toPandas()

,hack_license,pickup_datetime,dropoff_datetime,Pickup_borhood,dropoff_borhood,pickup_unix,dropoff_unix,lag_dropoff_unix,time_Between_trips
0,006313464EC98A24BB4EBC1E2419E439,2013-01-13 08:16:00,2013-01-13 08:20:00,Manhattan,Manhattan,1358064960,1358065200,1358063280,1680
1,006313464EC98A24BB4EBC1E2419E439,2013-01-13 08:51:00,2013-01-13 08:53:00,Manhattan,Manhattan,1358067060,1358067180,1358065200,1860
2,006313464EC98A24BB4EBC1E2419E439,2013-01-13 10:03:00,2013-01-13 10:07:00,Manhattan,Manhattan,1358071380,1358071620,1358067180,4200
3,006313464EC98A24BB4EBC1E2419E439,2013-01-13 11:31:00,2013-01-13 11:35:00,Manhattan,Manhattan,1358076660,1358076900,1358071620,5040
4,006313464EC98A24BB4EBC1E2419E439,2013-01-13 13:04:00,2013-01-13 13:31:00,Manhattan,Manhattan,1358082240,1358083860,1358076900,5340


In [0]:
#get the dataset count 
NYTaxi.selectExpr("*").count()

Out[73]: 87659

In [0]:
#exclude the rows with differnce time  more than 4 hours 
NYTaxi = NYTaxi.where("time_Between_trips < 14400 ").select('*')

In [0]:
#check the dataset row count 
NYTaxi.selectExpr("*").count()

Out[77]: 86128

In [0]:
pivotDF = NYTaxi.groupBy("dropoff_borhood").mean("time_Between_trips")

In [0]:
pivotDF.show()

+---------------+-----------------------+
dropoff_borhood|avg(time_Between_trips)|
+---------------+-----------------------+
 Queens| 1727.7096774193549|
 Brooklyn| 1612.744889937107|
 Staten Island| 1280.0|
 Manhattan| 982.007769201569|
 Bronx| 1639.7802197802198|
+---------------+-----------------------+